In [1]:
REMOVE_RES_CONTENT = False
# IMG_SIZE=299
IMG_SIZE=224
IMG_RESIZE = False
SEED = 42
RES_PATH = '/kaggle/working/res/'
DATASET_PATH = '/kaggle/input/deepfakedataset/data/'
WORKID_DIR_PATH='/kaggle/working/'

In [2]:
!pip install -q fairlearn tabulate
import zipfile
import pandas as pd
import numpy as np
import tabulate as tb
from typing import Dict
import tensorflow as tf
import re
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.applications import MobileNetV2
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference
from tensorflow.keras.applications import Xception, EfficientNetB0, ResNet50
from tensorflow.keras.callbacks import EarlyStopping
import os
import shutil
from sklearn.model_selection import train_test_split


np.random.seed(SEED)
tf.random.set_seed(SEED)


device_name = tf.test.gpu_device_name()
if device_name:
    print(f"GPU available: {device_name}")
else:
    print("No GPU available!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 6.3 MB/s eta 0:00:0000:01


2025-07-18 07:15:23.993364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752822924.232817      74 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752822924.297442      74 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU available: /device:GPU:0


I0000 00:00:1752822938.205222      74 gpu_device.cc:2022] Created device /device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752822938.206027      74 gpu_device.cc:2022] Created device /device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [3]:
file_path = DATASET_PATH + 'metadata.csv'
df_tmp = pd.read_csv(file_path, sep=',')
df_tmp['path'] = DATASET_PATH + df_tmp['path']

df_tmp = df_tmp[df_tmp['deepfake'] != 0]

df_tmp['ethnicity'] = df_tmp.apply(
    lambda row: 'white' if row['white'] == 1 else ('black' if row['black'] == 1 else (
        'asian' if row['asian'] == 1 else None)), axis=1)

df = df_tmp[['deepfake', 'male', 'ethnicity', 'eyeglasses', 'heavy_makeup', 'big_lips', 'path']]

df = df.rename(columns={'deepfake': 'type', 'male': 'sex', 'heavy_makeup': 'makeup', 'big_lips': 'lips',})

df['type'] = df['type'].replace({1: 'fake', -1: 'real'})
df['sex'] = df['sex'].replace({-1: 'female', 0: None, 1: 'male'})
df['makeup'] = df['makeup'].replace({-1: 'no', 0: None, 1: 'yes'})
df['lips'] = df['lips'].replace({-1: 'small', 0: None, 1: 'big'})
df['eyeglasses'] = df['eyeglasses'].replace({-1: 'no', 0: None, 1: 'yes'})


print(tb.tabulate(df.head(), headers='keys', tablefmt='psql'))
print(f"Dataset size: {len(df)}")

+----+--------+--------+-------------+--------------+----------+--------+--------------------------------------------------------------+
|    | type   | sex    | ethnicity   | eyeglasses   | makeup   | lips   | path                                                         |
|----+--------+--------+-------------+--------------+----------+--------+--------------------------------------------------------------|
|  0 | real   | male   |             | yes          | no       | big    | /kaggle/input/deepfakedataset/data/original/805/frame271.jpg |
|  1 | real   | female | white       | no           |          | big    | /kaggle/input/deepfakedataset/data/original/083/frame191.jpg |
|  2 | real   | male   | white       | no           | no       | small  | /kaggle/input/deepfakedataset/data/original/878/frame111.jpg |
|  3 | real   | female | white       | no           |          |        | /kaggle/input/deepfakedataset/data/original/158/frame201.jpg |
|  4 | real   | female | white       | no

In [4]:
def get_balanced_subset(
    df, class_col, feature_col, feature_value,
    samples_per_class, randomize=True, reset_index=False
):
    """
    Select a balanced subset of the data for a given feature value, with equal number of samples per class.

    Args:
        df: DataFrame
        class_col: column name of class labels
        feature_col: column name of feature
        feature_value: specific feature value to filter
        samples_per_class: number of samples per class
        randomize: whether to shuffle within class before selecting
        reset_index: whether to reset index of returned DataFrame
        seed: random seed for reproducibility

    Returns:
        Balanced DataFrame subset
    """
    tmp = df[df[feature_col] == feature_value]

    counts = tmp[class_col].value_counts()
    for cl, count in counts.items():
        if count < samples_per_class:
            raise ValueError(f"Not enough samples for class '{cl}' in feature '{feature_value}'. "
                             f"Required: {samples_per_class}, Available: {count}")

    tmp = pd.concat([
        (g.sample(frac=1, random_state=SEED).head(samples_per_class) if randomize else g.head(samples_per_class))
        for _, g in tmp.groupby(class_col)
    ])

    if reset_index:
        tmp = tmp.reset_index(drop=True)

    return tmp

tmp_test = get_balanced_subset(
    df=df, class_col='type', feature_col='sex', feature_value='male',
    samples_per_class=2, randomize=True, reset_index=True)
print(tb.tabulate(tmp_test, headers='keys', tablefmt='psql'))

+----+--------+-------+-------------+--------------+----------+--------+------------------------------------------------------------------+
|    | type   | sex   | ethnicity   | eyeglasses   | makeup   | lips   | path                                                             |
|----+--------+-------+-------------+--------------+----------+--------+------------------------------------------------------------------|
|  0 | fake   | male  | white       | no           | no       | small  | /kaggle/input/deepfakedataset/data/deepfake/374_407/frame41.jpg  |
|  1 | fake   | male  | white       | no           | no       | small  | /kaggle/input/deepfakedataset/data/deepfake/015_919/frame281.jpg |
|  2 | real   | male  |             | no           | no       | big    | /kaggle/input/deepfakedataset/data/original/995/frame11.jpg      |
|  3 | real   | male  | white       | no           | no       |        | /kaggle/input/deepfakedataset/data/original/579/frame201.jpg     |
+----+--------+-----

In [5]:
def get_exp_data(df, class_col, feature_col, ratio : Dict, size, randomize=True, exclude_column=None, exclude_df=None, max_diff=0.05):
    '''
    Get a balanced subset of the data based on specified ratios for features.
    Args:
        df: DataFrame containing the data
        class_col: column name for class labels
        feature_col: column name for features
        ratio: dictionary with feature values as keys and their ratios as values
        size: total number of samples to return
        randomize: whether to shuffle the DataFrame before processing
        exclude_column: column name to exclude from the DataFrame
        exclude_df: DataFrame containing values to exclude based on exclude_column
    '''
    if randomize:
        df_rnd = df.sample(frac=1, random_state=SEED).reset_index(drop=True)
    else:
        df_rnd = df.copy()

    if exclude_column is not None and exclude_df is not None:
        if exclude_column not in df_rnd.columns:
            raise ValueError(f"Column '{exclude_column}' not found in DataFrame.")
        if exclude_column not in exclude_df.columns:
            raise ValueError(f"Column '{exclude_column}' not found in exclude DataFrame.")
        df_rnd = df_rnd[~df_rnd[exclude_column].isin(exclude_df[exclude_column])]

    uniq_classes = df_rnd[class_col].unique()
    uniq_features = df_rnd[feature_col].unique()

    def get_exp_data_inner(tmp_df, size):
        df_tmp = None
        for uf in uniq_features:
            if ratio.get(uf) is None:
                print(f"Feature '{uf}' not found in ratios. Skipping.")
                continue
            c_amt = int(size * ratio[uf] / len(uniq_classes))
            # if c_amt <= 0:
            #     raise ValueError(f"Calculated samples per class ({c_amt}) is less than or equal to zero for feature '{uf}' with ratio {ratio}.")
            tmp = get_balanced_subset(df=tmp_df, class_col=class_col, feature_col=feature_col, feature_value=uf,
                                        samples_per_class=c_amt, randomize=False)
            if df_tmp is None:
                df_tmp = tmp
            else:
                df_tmp = pd.concat([df_tmp, tmp])
        return df_tmp

    df_res = get_exp_data_inner(df_rnd, size)

    if len(df_res) < size:
        print(f"Samples for ({len(df_res)}) are less than requested ({size}).")

    ratios_fet = df_res[feature_col].value_counts(normalize=True).to_dict()
    ratios_cls = df_res[class_col].value_counts(normalize=False).to_dict()
    print(f"[] Ratios for {feature_col}: {ratios_fet}")
    print(f"[] Ratios for {class_col}: {ratios_cls}")

    for k in ratio:
        if ratios_fet.get(k) is None:
            if ratio[k] > 0.0:
                raise ValueError(f"Feature '{k}' not found in DataFrame after sampling (try increase 'size' parameter).")
        elif abs(ratios_fet[k] - ratio[k]) > max_diff:
            raise ValueError(f"Feature '{k}' ratio {ratios_fet[k]} differs from requested {ratio[k]} by more than {max_diff}.")

    print()

    df_res = df_res.reset_index(drop=True)

    return df_res

tmp_test = get_exp_data(
    df=df, class_col='type', feature_col='ethnicity', ratio={'white':0.2, 'black':0.6, 'asian': 0.2}, size=10, randomize=True)
print(tb.tabulate(tmp_test, headers='keys', tablefmt='psql'))

Feature 'None' not found in ratios. Skipping.
[] Ratios for ethnicity: {'black': 0.6, 'white': 0.2, 'asian': 0.2}
[] Ratios for type: {'fake': 5, 'real': 5}

+----+--------+--------+-------------+--------------+----------+--------+------------------------------------------------------------------+
|    | type   | sex    | ethnicity   | eyeglasses   | makeup   | lips   | path                                                             |
|----+--------+--------+-------------+--------------+----------+--------+------------------------------------------------------------------|
|  0 | fake   | male   | white       | no           | no       | small  | /kaggle/input/deepfakedataset/data/deepfake/594_530/frame121.jpg |
|  1 | real   | female | white       | no           |          |        | /kaggle/input/deepfakedataset/data/original/240/frame41.jpg      |
|  2 | fake   | female | asian       |              |          | big    | /kaggle/input/deepfakedataset/data/deepfake/249_280/frame261.jp

In [6]:
def load_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    if IMG_RESIZE:
      image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image

def get_data_for_model(df, class_col, files_col, batch_size):
    image_paths = df[files_col].values
    labels = df[class_col].values
    labels = df[class_col].astype('category').cat.codes.values #classes strs to ints
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda path, label: (load_image(path), label))
    dataset = dataset.batch(batch_size)
    return dataset

In [7]:
def create_mobile_net2(num_classes, input_shape, model=None):
  if model is None:
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  else:
    for layer in model.layers:
        layer.trainable = True
    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
  return model, "mobile net 2"

def create_resnet50(num_classes, input_shape, model=None):
  if model is None:
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  else:
    for layer in model.layers:
      layer.trainable = True
    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
  return model, "resnet50"

def create_efficientnet_b0(num_classes, input_shape, model=None):
  if model is None:
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  else:
    for layer in model.layers:
      layer.trainable = True
    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

  return model, "efficientnet b0"


def create_xception(num_classes, input_shape, model=None):
  if model is None:
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  else:
    for layer in model.layers:
        layer.trainable = True
    model.compile(
        optimizer=Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

  return model, "xception"


In [8]:
# Create folder if it doesn't exist
if not os.path.exists(RES_PATH):
    os.makedirs(RES_PATH)
    print(f"Created folder: {RES_PATH}")
elif REMOVE_RES_CONTENT:
    # Remove all files inside the folder
    for filename in os.listdir(RES_PATH):
        file_path = os.path.join(RES_PATH, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)          # remove file or link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)      # remove folder and contents
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')
    print(f"Cleared contents of folder: {RES_PATH}")

Created folder: /kaggle/working/res/


In [9]:
def get_done_reps(model_name, feature_name, amt_per_rep):
  results_path = f'{RES_PATH}res_{feature_name}_{model_name.replace(" ", "_")}.csv'
  if not os.path.exists(results_path):
    return [], None

  tmp_df = pd.read_csv(results_path)
  dones = []
  for r in tmp_df["rep"].unique():
    amt = len(tmp_df[tmp_df["rep"]==r])
    if amt == amt_per_rep:
      dones.append(r)

  return dones, tmp_df

In [10]:
def perform_tests(df, train_metas, test_metas, validation_size, reps, class_col, feature_split_col, exclude_column, files_col, get_model, epochs_num_head, epochs_num_whole, batch_size):
    res = []

    _, model_name = get_model(num_classes=len(df[class_col].unique()), input_shape=(IMG_SIZE,IMG_SIZE,3))

    done_reps, prev_results = get_done_reps(model_name, feature_split_col, len(test_metas) * len(train_metas))

    for r in range(reps):
        if r in done_reps:
          res.extend(prev_results[prev_results['rep']==r].values.tolist())
          print(f"Rep {r} already done for {model_name}. Skipping...")
          continue

        np.random.seed(SEED + r)
        tf.random.set_seed(SEED + r)

        for train_meta in train_metas:
            train_val = get_exp_data(df, class_col=class_col, feature_col=feature_split_col, ratio=train_meta['ratio'], size=train_meta['size'] + validation_size)
            train_val = train_val.sample(frac=1, random_state=SEED+r).reset_index(drop=True)

            stratify_key = train_val[class_col].astype(str) + "_" + train_val[feature_split_col].astype(str)

            train, val = train_test_split(
                train_val,
                test_size=validation_size / (train_meta['size'] + validation_size),
                stratify=stratify_key,
                random_state=SEED + r
            )

            train = train.reset_index(drop=True)
            val = val.reset_index(drop=True)

            tests = [
                get_exp_data(df, class_col=class_col, feature_col=feature_split_col, ratio=tm['ratio'], size=tm['size'], exclude_column=exclude_column, exclude_df=train) for tm in test_metas
            ]

            train_dataset = get_data_for_model(train, class_col=class_col, files_col=files_col, batch_size=batch_size)
            val_dataset = get_data_for_model(val, class_col=class_col, files_col=files_col, batch_size=batch_size)
            test_datasets = [
                get_data_for_model(test, class_col=class_col, files_col=files_col, batch_size=batch_size) for test in tests
            ]

            train_ratio = '/'.join([f"{k}:{v}" for k, v in train_meta['ratio'].items()])
            train_ratio_rel = '/'.join([f"{k}:{v:.4f}" for k, v in train[feature_split_col].value_counts(normalize=True).to_dict().items()])
            train_ratio_sim = re.sub(r'[a-zA-Z0.:]', '', train_ratio)

            print("FITTING HEAD")
            model, model_name = get_model(num_classes=len(df[class_col].unique()), input_shape=(IMG_SIZE,IMG_SIZE,3))
            early_stopping_head = EarlyStopping(monitor='val_accuracy', patience=2, mode='max', restore_best_weights=True, verbose=1)
            model.fit(train_dataset, validation_data=val_dataset, epochs=epochs_num_head, callbacks=[early_stopping_head])

            print("FITTING WHOLE")
            model, model_name = get_model(num_classes=len(df[class_col].unique()), input_shape=(IMG_SIZE,IMG_SIZE,3), model=model)
            early_stopping_whole = EarlyStopping(monitor='val_accuracy', patience=2, mode='max', restore_best_weights=True, verbose=1)
            model.fit(train_dataset, validation_data=val_dataset, epochs=epochs_num_whole, callbacks=[early_stopping_whole])

            for test_dataset, test_meta, test_df in zip(test_datasets, test_metas, tests):
                predictions = model.predict(test_dataset)
                y_true = test_df[class_col].astype('category').cat.codes.values
                y_pred = np.argmax(predictions, axis=1)
                acc = accuracy_score(y_true, y_pred)
                f1 = f1_score(y_true, y_pred, average='weighted')
                eo_diff = equalized_odds_difference(y_true, y_pred, sensitive_features=test_df[feature_split_col])

                test_ratio = '/'.join([f"{k}:{v}" for k, v in test_meta['ratio'].items()])
                test_ratio_rel = '/'.join([f"{k}:{v:.4f}" for k, v in test_df[feature_split_col].value_counts(normalize=True).to_dict().items()])
                test_ratio_sim = re.sub(r'[a-zA-Z0.:]', '', test_ratio)

                res.append([
                    r,
                    model_name,
                    feature_split_col,
                    train_meta['size'],
                    train_ratio,
                    test_meta['size'],
                    test_ratio,
                    acc,
                    f1,
                    eo_diff,
                    train_ratio_rel,
                    test_ratio_rel,
                    train_ratio_sim,
                    test_ratio_sim
                ])

                print(f"Rep: {r:2} | Model: {model_name} | Feature Split: {feature_split_col} | Ratio: {test_ratio} | Acc: {acc:.2f}")

                res_df = pd.DataFrame(res, columns=[
                    'rep', 'model_name', 'feature_split_col',
                    'train_size', 'train_ratio_detail', 'test_size', 'test_ratio_detail',
                    'accuracy', 'f1_score', 'eo_diff', 'train_ratio_rel', 'test_ratio_rel', "train_ratio", "test_ratio"
                ])

                res_df.to_csv(f'{RES_PATH}res_{feature_split_col}_{model_name.replace(" ", "_")}.csv', index=False)
    print(f"Done for {model_name}.")


In [13]:

perform_tests(df=df,
              train_metas=[
                  {'ratio': {'male':0.1, 'female':0.9}, 'size': 5000},
                  {'ratio': {'male':0.5, 'female':0.5}, 'size': 5000},
                  ],
              test_metas=[
                  {'ratio': {'male':0.1, 'female':0.9}, 'size': 500},
                  {'ratio': {'male':0.3, 'female':0.7}, 'size': 500},
                  {'ratio': {'male':0.5, 'female':0.5}, 'size': 500},
                  {'ratio': {'male':0.7, 'female':0.3}, 'size': 500},
                  {'ratio': {'male':0.9, 'female':0.1}, 'size': 500},
              ],
              validation_size=500,
              reps=10,
              class_col='type',
              feature_split_col='sex',
              exclude_column='path',
              files_col='path',
              get_model=create_efficientnet_b0,
              epochs_num_head=8,
              epochs_num_whole=4,
              batch_size=64
              )

perform_tests(df=df,
              train_metas=[
                  {'ratio': {'male':0.1, 'female':0.9}, 'size': 5000},
                  {'ratio': {'male':0.5, 'female':0.5}, 'size': 5000},
                  ],
              test_metas=[
                  {'ratio': {'male':0.1, 'female':0.9}, 'size': 500},
                  {'ratio': {'male':0.3, 'female':0.7}, 'size': 500},
                  {'ratio': {'male':0.5, 'female':0.5}, 'size': 500},
                  {'ratio': {'male':0.7, 'female':0.3}, 'size': 500},
                  {'ratio': {'male':0.9, 'female':0.1}, 'size': 500},
              ],
              validation_size=500,
              reps=10,
              class_col='type',
              feature_split_col='sex',
              exclude_column='path',
              files_col='path',
              get_model=create_xception,
              epochs_num_head=8,
              epochs_num_whole=4,
              batch_size=64
              )

I0000 00:00:1752823007.180869      74 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752823007.181120      74 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Feature 'None' not found in ratios. Skipping.
[] Ratios for sex: {'female': 0.9, 'male': 0.1}
[] Ratios for type: {'fake': 2750, 'real': 2750}

Feature 'None' not found in ratios. Skipping.
[] Ratios for sex: {'female': 0.9, 'male': 0.1}
[] Ratios for type: {'fake': 250, 'real': 250}

Feature 'None' not found in ratios. Skipping.
[] Ratios for sex: {'female': 0.7, 'male': 0.3}
[] Ratios for type: {'fake': 250, 'real': 250}

Feature 'None' not found in ratios. Skipping.
[] Ratios for sex: {'female': 0.5, 'male': 0.5}
[] Ratios for type: {'fake': 250, 'real': 250}

Feature 'None' not found in ratios. Skipping.
[] Ratios for sex: {'male': 0.7, 'female': 0.3}
[] Ratios for type: {'fake': 250, 'real': 250}

Feature 'None' not found in ratios. Skipping.
[] Ratios for sex: {'male': 0.9, 'female': 0.1}
[] Ratios for type: {'fake': 250, 'real': 250}

FITTING HEAD
Epoch 1/8


I0000 00:00:1752823022.238285     133 service.cc:148] XLA service 0x7b7c0884df30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752823022.239425     133 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752823022.239445     133 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752823023.709645     133 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/79 ━━━━━━━━━━━━━━━━━━━━ 23:27 18s/step - accuracy: 0.5312 - loss: 0.7949

I0000 00:00:1752823030.056368     133 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


79/79 ━━━━━━━━━━━━━━━━━━━━ 43s 324ms/step - accuracy: 0.5021 - loss: 0.7314 - val_accuracy: 0.5180 - val_loss: 0.6957
Epoch 2/8
79/79 ━━━━━━━━━━━━━━━━━━━━ 12s 157ms/step - accuracy: 0.5004 - loss: 0.7025 - val_accuracy: 0.5000 - val_loss: 0.7054
Epoch 3/8
79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 162ms/step - accuracy: 0.5044 - loss: 0.7040 - val_accuracy: 0.5000 - val_loss: 0.7060
Epoch 4/8
79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 167ms/step - accuracy: 0.4964 - loss: 0.7025 - val_accuracy: 0.5020 - val_loss: 0.7053
Epoch 5/8
79/79 ━━━━━━━━━━━━━━━━━━━━ 14s 176ms/step - accuracy: 0.4971 - loss: 0.7007 - val_accuracy: 0.5020 - val_loss: 0.7043
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 1.
FITTING WHOLE
Epoch 1/4
79/79 ━━━━━━━━━━━━━━━━━━━━ 156s 966ms/step - accuracy: 0.5145 - loss: 0.8957 - val_accuracy: 0.5060 - val_loss: 0.7068
Epoch 2/4
79/79 ━━━━━━━━━━━━━━━━━━━━ 50s 628ms/step - accuracy: 0.8201 - loss: 0.4890 - val_accuracy: 0.5340 - val_loss: 0.6921
Epoch 3/4
79/79 

KeyboardInterrupt: 

In [ ]:
def print_summarise_res(model_name:str):
  path = RES_PATH + f'res_sex_{model_name}.csv'

  if not os.path.exists(path):
    print(f"File {path} does not exists!")
    return

  res = pd.read_csv(path)
  gr = res.groupby(['train_ratio', 'test_ratio']).agg(
      # Model=('model_name', 'first'),
      TrainRatio=('train_ratio', 'first'),
      TestRatio=('test_ratio', 'first'),
      Accuracy= ('accuracy', 'mean'),
      AccuracySTD= ('accuracy', 'std'),
      F1=('f1_score', 'mean'),
      F1STD=('f1_score', 'std'),
      EODiff=('eo_diff', 'mean'),
      EODiffSTD=('eo_diff', 'std'),
  ).reset_index(drop=True)

  gr = gr.round(3).sort_values(by=['TrainRatio', 'TestRatio'], ascending=False)

  print("MODEL: " + model_name)
  print(tb.tabulate(gr, headers='keys', tablefmt='psql'))
  print()
  print()

print_summarise_res('resnet50')
print_summarise_res('efficientnet_b0')
print_summarise_res('xception')